# PK Date & Time Reconciliation

In [2]:
import csv
import json
import pprint
import datetime
import os
import re
from matplotlib import pyplot

%matplotlib inline

## Setup Output Folder

_Global_ scope variables:

In [3]:
outputfolder = './test/safety/output'

Create the __output folder__ in which all output files will be placed.

In [7]:
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder, 0o755) # owner execution 

View contents of folder:

In [9]:
%ls -l ./test

total 0
drwxr-xr-x  2 herbsca  staff   64 17 Apr 11:24 output/
drwxr-xr-x  7 herbsca  staff  224 17 Apr 11:22 pk/
drwxr-xr-x  3 herbsca  staff   96 17 Apr 11:20 safety/


Define output file writing functions:

In [5]:
def writeFile(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(pprint.pformat(contents))

def writeJSON(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(json.dumps(contents))

## Create Mapping

Map csv file records to the csv file/subject records of medrio.

In [6]:
randfilename = './test/rand.csv'

subjectMap = dict() # dictionary for randomisation to subject id.

randCol = 'DARandNum_C' # column header for randomisation number
subjectCol = 'Subject ID' # column header for subject id

with open(randfilename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        subjectMap[f'{row[randCol]}'] = row[subjectCol]

writeFile('subjectMapping.txt', subjectMap)

## Read Configuration File

Context variables:

In [7]:
configfilename = './test/config.json'

Parse config file and establish relationships.

In [8]:
configDict = dict()

with open(configfilename) as jsonfile:
    configDict = json.load(jsonfile)

pprint.pp(configDict)

{'source': {'file': './INPUT.csv'},
 'comparison': {'file': './COMPARISON.csv'},
 'output': {'directory': './output'},
 'match': {'0': {'date': 'PKPDDat_C', 'time': 'PKPDTim_C'},
           '0.25': {'date': 'PK15Dat_C', 'time': 'PK15Tim_C'},
           '0.5': {'date': 'PK30Dat_C', 'time': 'PK30Tim_C'},
           '1': {'date': 'PK1Dat_C', 'time': 'PK1Tim_C'},
           '1.5': {'date': 'PK90Dat_C', 'time': 'PK90Tim_C'},
           '2': {'date': 'PK2Dat_C', 'time': 'PK2Tim_C'},
           '3': {'date': 'PKPM0Dat_C', 'time': 'PKPMTim_C'},
           '4': {'date': 'PK4Dat_C', 'time': 'PK4Tim_C'},
           '5': {'date': 'PK5Dat_C', 'time': 'PK5Tim_C'},
           '6': {'date': 'PK6Dat_C', 'time': 'PK6Tim_C'},
           '8': {'date': 'PKPM8Dat_C', 'time': 'PKPM8Tim_C'},
           '12': {'date': 'PK12Dat_C', 'time': 'PK12Tim_C'},
           '16': {'date': 'PK16Dat_C', 'time': 'PK16Tim_C'},
           '48': {'date': 'LB24Dat_C', 'time': 'LB24Tim_C'},
           '96': {'date': 'PK48Dat_C',

## Read Medrio Source File

Create dictionary mapping for each subject to allow for hash map search of data.

Context variables:

In [ ]:
sourcefilename = './test/source.csv'

sourceMap = dict() # dictionary for subject data

subjectidcol = 'Subject ID'
visitCol = 'Visit'

# Regular expression objects
dateregex = re.compile(r'dat', flags=re.I)
timeregex = re.compile(r'tim', flags=re.I)
periodRegex = re.compile(r'period.*([0-9])', flags=re.I)